# Coleta de Dados da API da Câmara dos Deputados

Este notebook realiza a coleta de dados da API da Câmara dos Deputados para análise de:

1. Presença dos deputados
2. Participação em votações
3. Comparação com regime CLT
4. Análise de remuneração efetiva
5. Comparação com trabalhadores civis

# 📘 coleta_dados
### **Objetivo:**
- Coletar lista de deputados e seus eventos via API da Câmara

In [1]:
import pandas as pd
import requests
from datetime import date
from pathlib import Path

# Configuração básica
BASE_URL = "https://dadosabertos.camara.leg.br/api/v2"
DATA_DIR = Path("../data/processed")
DATA_DIR.mkdir(parents=True, exist_ok=True)

In [2]:
# Função auxiliar simples para requests
def get_dados(endpoint, params=None):
    r = requests.get(f"{BASE_URL}/{endpoint}", params=params)
    r.raise_for_status()
    return r.json()["dados"]

# 1. Deputados e Presenças

In [4]:
# Coleta lista de deputados
params = {"ordem": "ASC", "ordenarPor": "nome"}
deputados = pd.DataFrame(get_dados("deputados", params))
deputados.to_json(DATA_DIR / "deputados.json", orient="records")

# 2. Eventos e Sessões

In [3]:
# Coleta eventos (sessões deliberativas)
from datetime import date
params = {
    "dataInicio": "2020-01-01",
    "dataFim": str(date.today()),
    "codTipoEvento": "1,2",  # Sessões deliberativas ordinárias e extraordinárias
    "ordem": "ASC",
    "ordenarPor": "dataHoraInicio"
}
eventos = pd.DataFrame(get_dados("eventos", params))
eventos.to_csv(DATA_DIR / "eventos_sessoes.csv", index=False)

# 3. Votações

In [24]:
# Coleta votações (2020-hoje) e salva mensalmente com resume automático.
import time, math, os, json, requests, pandas as pd
from pathlib import Path
from datetime import date
from tqdm import tqdm
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

BASE = "https://dadosabertos.camara.leg.br/api/v2"
DATA_DIR = Path("data/processed"); DATA_DIR.mkdir(parents=True, exist_ok=True)
HEADERS = {"Accept":"application/json","User-Agent":"PUCRS-TDE/1.0"}

def make_session():
    s = requests.Session()
    retry = Retry(
        total=5, connect=5, read=5, backoff_factor=1.2,
        status_forcelist=[400, 429, 500, 502, 503, 504],
        allowed_methods=["GET"]
    )
    s.mount("https://", HTTPAdapter(max_retries=retry))
    s.headers.update(HEADERS)
    return s

def meses(start="2020-01-01", end=str(date.today())):
    s, e = pd.to_datetime(start), pd.to_datetime(end)
    for m0 in pd.date_range(s, e, freq="MS"):
        m1 = (m0 + pd.offsets.MonthEnd(1))
        if m1 > e: m1 = e
        yield m0.strftime("%Y-%m"), m0.date().isoformat(), m1.date().isoformat()

def fetch_page(sess, endpoint, params):
    # timeout=(connect, read) evita pendurar indefinidamente
    r = sess.get(f"{BASE}/{endpoint}", params=params, timeout=(8, 40))
    if r.status_code == 400 and "ordenarPor" in params:
        # fallback sem ordenarPor quando dá 400
        p2 = dict(params); p2.pop("ordenarPor", None)
        r = sess.get(f"{BASE}/{endpoint}", params=p2, timeout=(8, 40))
    r.raise_for_status()
    j = r.json()
    return j.get("dados", [])

def coletar_votacoes_resumivel(di="2020-01-01", df=str(date.today()), itens=200):
    sess = make_session()
    for tag, ini, fim in tqdm(list(meses(di, df)), desc="Meses"):
        arq_mes = DATA_DIR / f"votacoes_{tag}.csv"
        if arq_mes.exists() and arq_mes.stat().st_size > 0:
            continue  # já coletado
        rows, pag = [], 1
        try:
            while True:
                params = {
                    "dataInicio": ini, "dataFim": fim,
                    "pagina": pag, "itens": itens, "ordem": "ASC",
                    "ordenarPor": "dataHoraRegistro"
                }
                dados = fetch_page(sess, "votacoes", params)
                if not dados: break
                rows.extend(dados)
                if len(dados) < itens: break
                pag += 1
                time.sleep(0.05)
        except KeyboardInterrupt:
            # salva parcial para não perder o mês
            if rows:
                pd.DataFrame(rows).to_csv(DATA_DIR / f"votacoes_{tag}__partial.csv", index=False)
            raise
        pd.DataFrame(rows).to_csv(arq_mes, index=False)
    # consolida (rápido e idempotente)
    vot_files = sorted(DATA_DIR.glob("votacoes_20*.csv"))
    df = pd.concat((pd.read_csv(p) for p in vot_files), ignore_index=True) if vot_files else pd.DataFrame()
    df.to_csv(DATA_DIR / "votacoes.csv", index=False)
    return df

def coletar_votos_resumivel(votacoes_df, itens=200):
    if votacoes_df.empty:
        return pd.DataFrame()
    votacoes_df["mes_tag"] = pd.to_datetime(votacoes_df["data"], errors="coerce").dt.strftime("%Y-%m")
    sess = make_session()
    for tag, grupo in votacoes_df.groupby("mes_tag", dropna=True):
        arq_mes = DATA_DIR / f"votos_{tag}.csv"
        if arq_mes.exists() and arq_mes.stat().st_size > 0:
            continue
        votos = []
        try:
            for vid in tqdm(grupo["id"].astype(str), desc=f"Votos {tag}"):
                pag = 1
                while True:
                    dados = fetch_page(sess, f"votacoes/{vid}/votos", {"pagina": pag, "itens": itens})
                    if not dados: break
                    votos.extend(dados)
                    if len(dados) < itens: break
                    pag += 1
                    time.sleep(0.02)
        except KeyboardInterrupt:
            if votos:
                pd.DataFrame(votos).to_csv(DATA_DIR / f"votos_{tag}__partial.csv", index=False)
            raise
        pd.DataFrame(votos).to_csv(arq_mes, index=False)
    # consolida
    v_files = sorted(DATA_DIR.glob("votos_20*.csv"))
    df = pd.concat((pd.read_csv(p) for p in v_files), ignore_index=True) if v_files else pd.DataFrame()
    df.to_csv(DATA_DIR / "votos_deputados.csv", index=False)
    return df

# ===== Execução =====
votacoes = coletar_votacoes_resumivel("2020-01-01", str(date.today()), itens=200)
votos_df = coletar_votos_resumivel(votacoes, itens=200)
print("OK:", len(votacoes), "votações;", len(votos_df), "votos.")


Meses: 100%|██████████| 70/70 [08:41<00:00,  7.46s/it]



EmptyDataError: No columns to parse from file

# 4. Presenças em Sessões

In [30]:
# Presenças em sessões — coleta total por ano (independe do DataFrame `eventos`)
import io, re, requests, pandas as pd
from pathlib import Path
from datetime import date

DATA_DIR = Path("data/processed"); DATA_DIR.mkdir(parents=True, exist_ok=True)
OUT = DATA_DIR / "presencas.csv"

BASE_ARQ = "http://dadosabertos.camara.leg.br/arquivos/eventosPresencaDeputados/csv/eventosPresencaDeputados-{ano}.csv"

# Anos a cobrir
anos = list(range(2020, date.today().year + 1))

def padronizar_presencas(df_raw: pd.DataFrame, ano: int) -> pd.DataFrame:
    # normaliza nomes
    cols_lower = {c.lower(): c for c in df_raw.columns}

    # coluna do id do evento
    id_evento_col = None
    for key in ("idevento","id_evento","evento_id","id"):
        if key in cols_lower: id_evento_col = cols_lower[key]; break
    if not id_evento_col:
        id_evento_col = next((c for c in df_raw.columns if "evento" in c.lower()), None)
    if not id_evento_col:
        raise KeyError(f"[{ano}] Não achei coluna de evento. Colunas: {list(df_raw.columns)}")

    # coluna do id do deputado
    id_dep_col = None
    for key in ("iddeputado","id_deputado","idparlamentar","idecadastro"):
        if key in cols_lower: id_dep_col = cols_lower[key]; break
    if not id_dep_col:
        id_dep_col = next((c for c in df_raw.columns if "deputad" in c.lower()), None)

    # coluna do tipo de presença
    tipo_col = None
    for key in ("tipopresenca","tipo_presenca"):
        if key in cols_lower: tipo_col = cols_lower[key]; break
    if not tipo_col:
        tipo_col = next((c for c in df_raw.columns if "presen" in c.lower() and "tipo" in c.lower()), None)

    out = pd.DataFrame()
    out["id_evento"] = pd.to_numeric(df_raw[id_evento_col].astype(str).str.extract(r"(\d+)")[0], errors="coerce").astype("Int64")
    out["id_deputado"] = (
        pd.to_numeric(df_raw[id_dep_col].astype(str).str.extract(r"(\d+)")[0], errors="coerce").astype("Int64")
        if id_dep_col else pd.NA
    )
    out["tipo_presenca"] = df_raw[tipo_col] if tipo_col else pd.NA
    out["ano_origem"] = ano
    out = out.dropna(subset=["id_evento"])
    return out

parts = []
for ano in anos:
    url = BASE_ARQ.format(ano=ano)
    r = requests.get(url, timeout=60)
    if r.status_code == 404:
        # pode não existir para anos sem dados
        continue
    r.raise_for_status()
    buf = io.BytesIO(r.content)
    # CSVs do portal costumam usar ';' e encoding UTF-8
    df_raw = pd.read_csv(buf, sep=';', dtype=str, low_memory=False)
    parts.append(padronizar_presencas(df_raw, ano))

presencas = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame(columns=["id_evento","id_deputado","tipo_presenca","ano_origem"])
presencas = presencas.drop_duplicates(subset=["id_evento","id_deputado","tipo_presenca"])

presencas.to_csv(OUT, index=False)
print(f"OK — presenças salvas em {OUT} | linhas: {len(presencas)}")

# Dica: se depois quiser filtrar pelos seus eventos específicos, faça:
# eventos = pd.read_csv(DATA_DIR/'eventos_sessoes.csv')  # por ex.
# presencas = presencas[presencas['id_evento'].isin(eventos['id_evento'])]
# presencas.to_csv(OUT, index=False)


OK — presenças salvas em data/processed/presencas.csv | linhas: 564081


# 5. Dados de Remuneração

In [19]:
# Coleta detalhes profissionais
remuneracoes = []
for id_dep in deputados["id"]:
    dados = get_dados(f"deputados/{id_dep}")
    remuneracoes.append({
        "id_deputado": id_dep,
        "cargo": dados.get("ultimoStatus", {}).get("condicaoEleitoral"),
        "situacao": dados.get("ultimoStatus", {}).get("situacao"),
        "data_inicio": dados.get("ultimoStatus", {}).get("data"),
    })
pd.DataFrame(remuneracoes).to_csv(DATA_DIR / "remuneracoes.csv", index=False)

### 1️⃣ Deputados:
- Geramos **deputados.json** - id, uri, nome, siglaPartido, uriPartido, siglaUF, idLegislatura, urlFoto e email


In [20]:
print("Coletando lista de deputados...")
deputados = []
pagina = 1

while True:
    params = {
        "itens": 100,
        "pagina": pagina,
        "ordem": "ASC",
        "ordenarPor": "nome"
    }
    
    dados = make_request(f"{BASE_URL}/deputados", params)
    if not dados:
        break
        
    deputados.extend(dados)
    pagina += 1
    time.sleep(0.1)  # polidez

# Salvando dados
ARQ_DEPUTADOS.write_text(
    json.dumps(deputados, ensure_ascii=False, indent=2),
    encoding="utf-8"
)

print(f"✅ Deputados coletados: {len(deputados)} → {ARQ_DEPUTADOS}")

Coletando lista de deputados...
✅ Deputados coletados: 513 → ../data/processed/deputados.json
✅ Deputados coletados: 513 → ../data/processed/deputados.json


### OBS: Frequencia Eventos:
- Geramos **freq_eventos.csv** - id_deputado e num_eventos


In [ ]:
# Carregar IDs dos deputados
with open(ARQ_DEPUTADOS, "r", encoding="utf-8") as f:
    ids_deputados = [int(d["id"]) for d in json.load(f)]

# Preparar CSV de saída
with open(ARQ_FREQ, "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerow(["id_deputado", "num_eventos"])

print("Coletando eventos por deputado...")
total_eventos = 0

for i, dep_id in enumerate(ids_deputados, 1):
    total_dep = 0
    pagina = 1
    
    while True:
        params = {
            "dataInicio": DATA_INICIO,
            "dataFim": DATA_FIM,
            "itens": 100,
            "ordenarPor": "dataHoraInicio",
            "ordem": "ASC",
            "pagina": pagina
        }
        
        dados = make_request(f"{BASE_URL}/deputados/{dep_id}/eventos", params)
        if not dados:
            break
            
        total_dep += len(dados)
        pagina += 1
        time.sleep(0.1)  # polidez
    
    # Gravar contagem do deputado
    with open(ARQ_FREQ, "a", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow([dep_id, total_dep])
    
    total_eventos += total_dep
    
    # Log de progresso
    if i % 25 == 0 or i == len(ids_deputados):
        print(f"{i}/{len(ids_deputados)} deputados processados... Total eventos = {total_eventos}")

print(f"✅ Contagens geradas → {ARQ_FREQ}")
print(f"📊 Total de eventos coletados: {total_eventos}")

25/513 deputados processados… acumulado eventos = 22793
50/513 deputados processados… acumulado eventos = 43357
50/513 deputados processados… acumulado eventos = 43357
75/513 deputados processados… acumulado eventos = 73656
75/513 deputados processados… acumulado eventos = 73656
100/513 deputados processados… acumulado eventos = 98560
100/513 deputados processados… acumulado eventos = 98560
125/513 deputados processados… acumulado eventos = 119013
125/513 deputados processados… acumulado eventos = 119013
150/513 deputados processados… acumulado eventos = 140355
150/513 deputados processados… acumulado eventos = 140355
175/513 deputados processados… acumulado eventos = 163215
175/513 deputados processados… acumulado eventos = 163215
200/513 deputados processados… acumulado eventos = 186858
200/513 deputados processados… acumulado eventos = 186858
225/513 deputados processados… acumulado eventos = 209412
225/513 deputados processados… acumulado eventos = 209412
250/513 deputados processa

## 4 Presenças em sessões:

In [29]:
# Presenças em sessões — coleta total por ano (independe do DataFrame `eventos`)
import io, re, requests, pandas as pd
from pathlib import Path
from datetime import date

DATA_DIR = Path("data/processed"); DATA_DIR.mkdir(parents=True, exist_ok=True)
OUT = DATA_DIR / "presencas.csv"

BASE_ARQ = "http://dadosabertos.camara.leg.br/arquivos/eventosPresencaDeputados/csv/eventosPresencaDeputados-{ano}.csv"

# Anos a cobrir
anos = list(range(2020, date.today().year + 1))

def padronizar_presencas(df_raw: pd.DataFrame, ano: int) -> pd.DataFrame:
    # normaliza nomes
    cols_lower = {c.lower(): c for c in df_raw.columns}

    # coluna do id do evento
    id_evento_col = None
    for key in ("idevento","id_evento","evento_id","id"):
        if key in cols_lower: id_evento_col = cols_lower[key]; break
    if not id_evento_col:
        id_evento_col = next((c for c in df_raw.columns if "evento" in c.lower()), None)
    if not id_evento_col:
        raise KeyError(f"[{ano}] Não achei coluna de evento. Colunas: {list(df_raw.columns)}")

    # coluna do id do deputado
    id_dep_col = None
    for key in ("iddeputado","id_deputado","idparlamentar","idecadastro"):
        if key in cols_lower: id_dep_col = cols_lower[key]; break
    if not id_dep_col:
        id_dep_col = next((c for c in df_raw.columns if "deputad" in c.lower()), None)

    # coluna do tipo de presença
    tipo_col = None
    for key in ("tipopresenca","tipo_presenca"):
        if key in cols_lower: tipo_col = cols_lower[key]; break
    if not tipo_col:
        tipo_col = next((c for c in df_raw.columns if "presen" in c.lower() and "tipo" in c.lower()), None)

    out = pd.DataFrame()
    out["id_evento"] = pd.to_numeric(df_raw[id_evento_col].astype(str).str.extract(r"(\d+)")[0], errors="coerce").astype("Int64")
    out["id_deputado"] = (
        pd.to_numeric(df_raw[id_dep_col].astype(str).str.extract(r"(\d+)")[0], errors="coerce").astype("Int64")
        if id_dep_col else pd.NA
    )
    out["tipo_presenca"] = df_raw[tipo_col] if tipo_col else pd.NA
    out["ano_origem"] = ano
    out = out.dropna(subset=["id_evento"])
    return out

parts = []
for ano in anos:
    url = BASE_ARQ.format(ano=ano)
    r = requests.get(url, timeout=60)
    if r.status_code == 404:
        # pode não existir para anos sem dados
        continue
    r.raise_for_status()
    buf = io.BytesIO(r.content)
    # CSVs do portal costumam usar ';' e encoding UTF-8
    df_raw = pd.read_csv(buf, sep=';', dtype=str, low_memory=False)
    parts.append(padronizar_presencas(df_raw, ano))

presencas = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame(columns=["id_evento","id_deputado","tipo_presenca","ano_origem"])
presencas = presencas.drop_duplicates(subset=["id_evento","id_deputado","tipo_presenca"])

presencas.to_csv(OUT, index=False)
print(f"OK — presenças salvas em {OUT} | linhas: {len(presencas)}")

# Dica: se depois quiser filtrar pelos seus eventos específicos, faça:
# eventos = pd.read_csv(DATA_DIR/'eventos_sessoes.csv')  # por ex.
# presencas = presencas[presencas['id_evento'].isin(eventos['id_evento'])]
# presencas.to_csv(OUT, index=False)


OK — presenças salvas em data/processed/presencas.csv | linhas: 564081


### Situação parlamentar:

In [ ]:
import os, time, json, csv, requests
from datetime import date

BASE_URL = "https://dadosabertos.camara.leg.br/api/v2"
DIR_SAIDA = "../data/processed"
ARQ_DEPUTADOS = f"{DIR_SAIDA}/deputados.json"
ARQ_FREQ = f"{DIR_SAIDA}/freq_eventos.csv"

os.makedirs(DIR_SAIDA, exist_ok=True)

# janela temporal
DATA_INICIO = "2020-01-01"
DATA_FIM = str(date.today())  # hoje

# sessão HTTP enxuta
sessao = requests.Session()
sessao.headers.update({
    "Accept": "application/json",
    "User-Agent": "PUCRS-Assiduidade/1.0 (contato: seu_email@exemplo.com)"
})


r = sessao.get(f"{BASE_URL}/referencias/deputados/codSituacao")
situacoes = r.json()["dados"]
print(json.dumps(situacoes, indent=2, ensure_ascii=False))


[
  {
    "cod": "",
    "sigla": "A",
    "nome": "Afastado",
    "descricao": ""
  },
  {
    "cod": "",
    "sigla": "C",
    "nome": "Convocado",
    "descricao": ""
  },
  {
    "cod": "",
    "sigla": "E",
    "nome": "Exercício",
    "descricao": ""
  },
  {
    "cod": "",
    "sigla": "F",
    "nome": "Fim de Mandato",
    "descricao": ""
  },
  {
    "cod": "",
    "sigla": "L",
    "nome": "Licença",
    "descricao": ""
  },
  {
    "cod": "",
    "sigla": "S",
    "nome": "Suplência",
    "descricao": ""
  },
  {
    "cod": "",
    "sigla": "U",
    "nome": "Suspenso",
    "descricao": ""
  },
  {
    "cod": "",
    "sigla": "V",
    "nome": "Vacância",
    "descricao": ""
  }
]


### Ocupações por ID
- Geramos **ocupacoes.csv** - id_deputado, titulo


In [ ]:
print("Coletando ocupações dos deputados...")
# Preparar CSV de ocupações
with open(ARQ_OCUPACOES, "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerow(["id_deputado", "titulo", "dataInicio", "dataFim"])

# Carregar IDs dos deputados
with open(ARQ_DEPUTADOS, "r", encoding="utf-8") as f:
    ids_deputados = [int(d["id"]) for d in json.load(f)]

total_ocupacoes = 0
for i, dep_id in enumerate(ids_deputados, 1):
    dados = make_request(f"{BASE_URL}/deputados/{dep_id}/ocupacoes")
    if dados:
        with open(ARQ_OCUPACOES, "a", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            for o in dados:
                w.writerow([
                    dep_id,
                    o.get("titulo"),
                    o.get("dataInicio"),
                    o.get("dataFim")
                ])
                total_ocupacoes += 1
    
    if i % 25 == 0:
        print(f"... {i}/{len(ids_deputados)} deputados processados")
    time.sleep(0.1)

print(f"✅ Ocupações coletadas: {total_ocupacoes} → {ARQ_OCUPACOES}")